In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import time
import pandas as pd
import re
from openpyxl import load_workbook
from selenium import webdriver
from datetime import date

D:\Users\o.lukasevych\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def ele():
# получение всех ссылок rabota.ua на одной странице
    elems = driver.find_elements_by_xpath("//a[@href]")
    elems= [elem.get_attribute("href")for elem in elems]
    ln_links = [x for x in elems if re.search(r'^https://rabota.ua/cv', x)]
    return ln_links

In [3]:
# парсим с резюме нужные значения, записываем по-строчно в файл resumes.xlsx 
def rabotaua(link):
    name=None
    city=None
    age=None
    price=None
    brief=None
    exp=None
    edu=None
    lang=None
    
    
    response=requests.get(link)
    soup=BeautifulSoup(response.text,'html.parser')
    name=soup.find('span',id='ctl00_centerZone_BriefResume1_CvView1_cvHeader_lblName').text
    
    short=soup.find('p',{'class':'rua-p-t_12'}).text.strip().split('\r\n')
    city=short[0]
    regex = re.compile('\d+')
    age=regex.findall(short[1])[0]
    try:
        regex = re.compile('\d+')
        price=regex.findall(short[2])[0]+regex.findall(short[2])[1]
    except:
        pass
    try:
        brief=soup.find('span',id='ctl00_centerZone_BriefResume1_CvView1_Skills_txtSkills').text.replace('Программное обеспечение и сервисы: ','')
    except:
        pass
    exp=soup.find('div',id='ExperienceHolder').text.replace('Опыт работы','').replace('Нет опыта работы','')
    edu=soup.find('div',id='EducationHolder').text.replace('Образование','')
    lang=soup.find('div',id='LanguagesHolder').text.replace('Владение языками','')
    scrap_date=date.today()
    field1=[name,link,city,age," "," ",price,brief,exp,edu,lang,scrap_date]
    file_name = "resumes.xlsx"
    wb = load_workbook(file_name)

    page = wb.active
    page.append(field1)
    wb.save(file_name)
    name=None
    link=None
    city=None
    age=None
    price=None
    brief=None
    exp=None
    edu=None
    lang=None

In [4]:
# загружаем драйвер, открывается новое окно хрома, в этом окне делаем запрос на поиск, с каждой страницы
# собираем нужные ссылки, закрываем драйвер
driver = webdriver.Chrome('C:\\ProgramData\chromedriver.exe')

In [5]:
# google search 
# start driver (new chrome window)
driver.get('https:www.google.com')
time.sleep(3)

# запрос на поиск 
search_query = driver.find_element_by_name('q')
search_query.send_keys('site:rabota.ua/cv/ AND "коммерческий директор" AND "Киев" AND "FMCG"')
time.sleep(0.5)

# имитация ввода запроса на поиск
search_query.send_keys(u'\ue007')
time.sleep(3)

# формируем список ссылок с первых десяти страниц
list_link=[]
for i in range(4):
    list_link.append(ele())
    next_page=driver.find_element_by_xpath("//*[contains(local-name(),'span') and contains(text(),'Уперед')]")
    next_page.click()
    time.sleep(5)
driver.quit()

In [6]:
list_link

[['https://rabota.ua/cv/5871428',
  'https://rabota.ua/cv/6687243',
  'https://rabota.ua/cv/4546874',
  'https://rabota.ua/cv/9708136',
  'https://rabota.ua/cv/12024097',
  'https://rabota.ua/cv/11213338',
  'https://rabota.ua/cv/8302546',
  'https://rabota.ua/cv/6777503',
  'https://rabota.ua/cv/6316501',
  'https://rabota.ua/cv/6058623'],
 ['https://rabota.ua/cv/6240017',
  'https://rabota.ua/cv/9514211',
  'https://rabota.ua/cv/7032926',
  'https://rabota.ua/cv/4637418',
  'https://rabota.ua/cv/13856362',
  'https://rabota.ua/cv/5392396',
  'https://rabota.ua/cv/10838152',
  'https://rabota.ua/cv/13855279',
  'https://rabota.ua/cv/9210147',
  'https://rabota.ua/cv/5816323'],
 ['https://rabota.ua/cv/9165161',
  'https://rabota.ua/cv/6510369',
  'https://rabota.ua/cv/10372671',
  'https://rabota.ua/cv/13354079',
  'https://rabota.ua/cv/3366909',
  'https://rabota.ua/cv/5769251',
  'https://rabota.ua/cv/6558205',
  'https://rabota.ua/cv/7223610',
  'https://rabota.ua/cv/8012604',
  'ht

In [7]:
list_link=sum(list_link, [])# делаем

In [42]:
# все полученные ссылки исполняем функцией rabotaua(link)
link_drop=[]
for link in list_link:
    try:
        rabotaua(link)
        time.sleep(10)
    except:
        link_drop.append(link)
        pass

In [43]:
# ссылки, которые не сработали
link_drop

['https://rabota.ua/cv/6777503',
 'https://rabota.ua/cv/9165161',
 'https://rabota.ua/cv/7486792']

In [24]:
link=list_link[0]

In [25]:
name=None
city=None
age=None
price=None
brief=None
exp=None
edu=None
lang=None

In [26]:
response=requests.get(link)
response

<Response [200]>

In [27]:
soup=BeautifulSoup(response.text,'html.parser')

In [28]:
name=soup.find('span',id='ctl00_centerZone_BriefResume1_CvView1_cvHeader_lblName').text
#ctl00_centerZone_BriefResume1_CvView1_cvHeader_lblName
name


'Владимир'

In [29]:
short=soup.find('p',{'class':'rua-p-t_12'}).text.strip().split('\r\n')
city=short[0]
city

'Киев'

In [30]:
regex = re.compile('\d+')
age=regex.findall(short[1])[0]
age

'52'

In [33]:
try:
    regex = re.compile('\d+')
    price=regex.findall(short[2])[0]+regex.findall(short[2])[1]
except:
    pass

In [34]:
brief=soup.find('span',id='ctl00_centerZone_BriefResume1_CvView1_Skills_txtSkills').text.replace('Программное обеспечение и сервисы: ','')
brief
#ctl00_centerZone_BriefResume1_CvView1_Skills_txtSkills

'Владею навыками:организации и управления продажами продукции FMCG на территории Украины;стратегического планирования, создания структуры, бизнес-процессов и полного управления продажами и\xa0торговым предприятием;административно-хозяйственного управления предприятием;работы с дистрибьюторами и\xa0собственными торговыми командами;работы со скоропортом и продукцией, требующей специальных условий хранения.\xa0\xa0Достижения:2014-2016 - предприятие вошло в\xa0тройку лидеров производителей кондитерской продукции Украины (была шестая позиция);2010-2014 - организовал продажи\xa0абсолютно нового продукта, который\xa0принёс прибыль, превышающую прибыль от основного продукта\xa0предприятия;2005-2007 - организовал дистрибьюцию\xa0продукта с 0, что позволило предприятию стать лидером национального рынка.\xa0'

In [35]:
exp=soup.find('div',id='ExperienceHolder').text.replace('Опыт работы','').replace('Нет опыта работы','')
exp

'\r\nКоммерческий директорсен 2018 - настоящее время (1 год 3 мес)ООО "Житомирский молочный завод"FMCG\r\n\t\t\t\tУправляю коммерческой службой - закупки, продажи, логистика.Директор по продажамокт 2014 - настоящее время (5 лет 2 мес)ОДО "ЖЛ", ТМ "Житомирские ласощи"(http://sladosti.com.ua)\r\n\t\t\t\tболее 500 сотрудников\r\n\t\t\t\t\tFMCG\r\n\t\t\t\tУправляю продажами компании на территории Украины: - участвую в стратегическом планировании развития компании; - разрабатываю стратегию развития продаж в Украине; - разрабатываю планы продаж в Украине, контролирую выполнение; - разрабатываю торговые условия компани;  - руковожу торговым персоналом компании; - управляю дебиторской задолженностью;  - разрабатываю и управляю мероприятиями продвижения продукции;  - участвую в разработке новых видов продукции; - отчитываюсь перед инвесторами по финансовым результатам.\r\n\t\t\t\r\n\t\t\r\n\t\t\tДиректор компании. Директор департамента дистрибьюции группы компанийиюл 2010 - июн 2014 (4 года)Укр

In [36]:
edu=soup.find('div',id='EducationHolder').text.replace('Образование','')
edu

'Киевский индустриально-педагогический колледж, Технология сварочного производства, Киев.(г.Киев, Украина)\r\n\t\t\tГод окончания\r\n\t\t\t\t1993\r\n\t\t\tинженер-сварщик,\xa0специалист\r\n\t\r\n\r\n\t\t\t'

In [37]:
lang=soup.find('div',id='LanguagesHolder').text.replace('Владение языками','')
lang

'\n'

In [38]:
scrap_date=date.today()
scrap_date

datetime.date(2019, 11, 1)

In [39]:
field1=[name,link,city,age,price,brief,exp,edu,lang,scrap_date]
field1

['Владимир',
 'https://rabota.ua/cv/5871428',
 'Киев',
 '52',
 None,
 'Владею навыками:организации и управления продажами продукции FMCG на территории Украины;стратегического планирования, создания структуры, бизнес-процессов и полного управления продажами и\xa0торговым предприятием;административно-хозяйственного управления предприятием;работы с дистрибьюторами и\xa0собственными торговыми командами;работы со скоропортом и продукцией, требующей специальных условий хранения.\xa0\xa0Достижения:2014-2016 - предприятие вошло в\xa0тройку лидеров производителей кондитерской продукции Украины (была шестая позиция);2010-2014 - организовал продажи\xa0абсолютно нового продукта, который\xa0принёс прибыль, превышающую прибыль от основного продукта\xa0предприятия;2005-2007 - организовал дистрибьюцию\xa0продукта с 0, что позволило предприятию стать лидером национального рынка.\xa0',
 '\r\nКоммерческий директорсен 2018 - настоящее время (1 год 3 мес)ООО "Житомирский молочный завод"FMCG\r\n\t\t\t\tУпра

In [41]:
file_name = "resumes.xlsx"
wb = load_workbook(file_name)

page = wb.active
page.append(field1)
wb.save(file_name)